In [1]:
# Import Python Modules

import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import cartopy.crs as ccrs

from cartopy.feature import ShapelyFeature
from datetime import datetime
from shapely.geometry import Polygon

import h3, jenkspy

In [10]:
# Display Settings

# Enable the view of all columns
pd.set_option("display.max_columns", None) 

# Enable the view of all rows
pd.set_option("display.max_rows", None)

In [28]:
# Define Global Variables

# ACLED dataset
ukraine_acled_geopandas = gpd.read_file(r"D:\01 - CPD\07  -  MSc GIS\04 - Year 1\EGM722 Programming for GIS and Remote Sensing\Data\ACLED\UKR_ACLED_MSc.shp")

# Ukraine country boundary
ukraine_boundary_geopandas = gpd.read_file(r"D:\01 - CPD\07  -  MSc GIS\04 - Year 1\EGM722 Programming for GIS and Remote Sensing\Data\Boundaries\UKR_Boundary.shp")

# Ukraine oblast bondaries
ukraine_oblast_geopandas = gpd.read_file(r"D:\01 - CPD\07  -  MSc GIS\04 - Year 1\EGM722 Programming for GIS and Remote Sensing\Data\Boundaries\UKR_Adm.shp")

# Global country boudnaries
global_boundary_geopandas = gpd.read_file(r"D:\01 - CPD\07  -  MSc GIS\04 - Year 1\EGM722 Programming for GIS and Remote Sensing\Data\Boundaries\GLB_Bnd.shp")

# Start, middle and end dates
date_start, date_mid, date_end = ((datetime(2021,12,1), datetime(2022,2,1), datetime(2022,3,18)))

# ACLED dataset columns to keep (date will be calculated later on)
columns_acled = ["actor1", "ADMINCEN1", "event_date", "date", "fatalities", "geometry"]

In [32]:
def spatial_join_oblast(oblasts, acled):
    
    """
    Spatially join the ACLED dataset to country administrative boundaries

    Parameters : 

        argument 1 (geopandas dataframe) : Geopandas dataframe of the country boundaries dataset
    
        argument 2 (geopandas dataframe) : Geopandas dataframe of the ACLED dataset

    Returns : 

        argument 1 + 2 spaitally joined (geopandas dataframe) 
    """
    # Create new column for datetime timestamp attributes 
    
    acled_oblast_sj = gpd.sjoin(oblasts, acled, how='inner', lsuffix='left', rsuffix='right')
    
    return acled_oblast_sj

In [33]:
def convert_to_datetime(date):
    """
    Convert string date into a datetime object

    Parameters : 
    
        argument 1 (string) : Date to be converted, formatted as "02 March 2022"

    Returns : 

        datetime(argument 1) 
    """
    return datetime.strptime(date, "%d %B %Y")

In [34]:
def calculate_datetime_ranges(acled, columns, start, mid, end):
    
    """
    Create 3 new acled dataframes for the speicifed date ranges, keeping only those columns of interest

    Parameters : 
    
        argument 1 (geopandas dataframe) : Geopandas dataframe of the ACLED dataset

        argument 2 (list) : List of the columns to keep in with the dataframe

        argument 3 (datetime) : date of the start date

        argument 4 (datetime) : date of the middle date

        argument 5 (datetime) : date of the end date

    Returns : 

        geopandas dataframe : Date range start - end

        geopandas dataframe : Date range start - mid
        
        geopandas dataframe : Date range mid - end
    """

    # Create a new column called "date" and convert the original string dates into datetime
    acled["date"] = acled["event_date"].apply(convert_to_datetime)

    # Create geopandas dataframes for date range start - end
    acled_daterange_se = acled[(acled["date"] >= start) & (acled["date"] <= end)]

    # Create geopandas dataframes for date range start - mid
    acled_daterange_sm = acled[(acled["date"] < mid) & (acled["date"] >= start)]
    
    # Create geopandas dataframes for date range mid - end
    acled_daterange_me = acled[(acled["date"] >= mid) & (acled["date"] <= end)]
    
    return acled_daterange_se[columns], acled_daterange_sm[columns], acled_daterange_me[columns]


In [35]:
# Spatially join ACLED to country oblasts
acled_oblast_sj = spatial_join_oblast(ukraine_oblast_geopandas, ukraine_acled_geopandas)

# Tuple unpack returned dataframes
acled_daterange_se, acled_daterange_sm, acled_daterange_me = calculate_datetime_ranges(acled_oblast_sj, columns_acled, date_start, date_mid, date_end)

In [36]:
acled_daterange_se.head()

,actor1,ADMINCEN1,event_date,date,fatalities,geometry
0,Police Forces of Ukraine (2019-),Kyiv,02 March 2022,2022-03-02,0,"POLYGON ((30.25215 50.48641, 30.25238 50.48637..."
0,Protesters (Ukraine),Kyiv,19 February 2022,2022-02-19,0,"POLYGON ((30.25215 50.48641, 30.25238 50.48637..."
0,Protesters (Ukraine),Kyiv,28 December 2021,2021-12-28,0,"POLYGON ((30.25215 50.48641, 30.25238 50.48637..."
0,Protesters (Ukraine),Kyiv,28 December 2021,2021-12-28,0,"POLYGON ((30.25215 50.48641, 30.25238 50.48637..."
0,Military Forces of Ukraine (2019-) Air Force,Kyiv,07 March 2022,2022-03-07,0,"POLYGON ((30.25215 50.48641, 30.25238 50.48637..."
